In [ ]:
from IPython import get_ipython
from IPython.display import display
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import os
import shutil


In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from tensorflow.keras.applications import ResNet50

In [ ]:
conv_base = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

In [ ]:
conv_base.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
conv_base.trainable = True
set_trainable = False

for layer in conv_base.layers:
    if layer.name == "conv5_block1_out":
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

for layer in conv_base.layers:
  print(layer.name,layer.trainable)

input_layer False
conv1_pad False
conv1_conv False
conv1_bn False
conv1_relu False
pool1_pad False
pool1_pool False
conv2_block1_1_conv False
conv2_block1_1_bn False
conv2_block1_1_relu False
conv2_block1_2_conv False
conv2_block1_2_bn False
conv2_block1_2_relu False
conv2_block1_0_conv False
conv2_block1_3_conv False
conv2_block1_0_bn False
conv2_block1_3_bn False
conv2_block1_add False
conv2_block1_out False
conv2_block2_1_conv False
conv2_block2_1_bn False
conv2_block2_1_relu False
conv2_block2_2_conv False
conv2_block2_2_bn False
conv2_block2_2_relu False
conv2_block2_3_conv False
conv2_block2_3_bn False
conv2_block2_add False
conv2_block2_out False
conv2_block3_1_conv False
conv2_block3_1_bn False
conv2_block3_1_relu False
conv2_block3_2_conv False
conv2_block3_2_bn False
conv2_block3_2_relu False
conv2_block3_3_conv False
conv2_block3_3_bn False
conv2_block3_add False
conv2_block3_out False
conv3_block1_1_conv False
conv3_block1_1_bn False
conv3_block1_1_relu False
conv3_block1_2

In [ ]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(1024,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │     102,761,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 127,924,097 (487.99 MB)

 Trainable params: 113,267,713 (432.08 MB)

 Non-trainable params: 14,656,384 (55.91 MB)

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Images.zip" -d "/content/drive/MyDrive/Image"

unzip:  cannot find or open /content/drive/MyDrive/Images.zip, /content/drive/MyDrive/Images.zip.zip or /content/drive/MyDrive/Images.zip.ZIP.


In [ ]:
import os
folder_path = '/content/drive/MyDrive/Folder/'
if os.path.exists(folder_path):
  print(f"The folder '{folder_path}' exists.")
else:
  print("The folder not exists.")

The folder '/content/drive/MyDrive/Folder/' exists.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 4
image_size = (224,224)
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=20,  # Random rotations
    width_shift_range=0.2,  # Horizontal shifts
    height_shift_range=0.2,  # Vertical shifts
    shear_range=0.15,  # Shearing
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Randomly flip images
    fill_mode='nearest', # Filling strategy for missing pixels after transformation
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Folder/wildfire_images_split/train',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode = 'binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Folder/wildfire_images_split/test',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 1271 images belonging to 2 classes.
Found 318 images belonging to 2 classes.


In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-5), loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator ,epochs=20,validation_data=validation_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
318/318 ━━━━━━━━━━━━━━━━━━━━ 587s 2s/step - accuracy: 0.6112 - loss: 0.7380 - val_accuracy: 0.3805 - val_loss: 0.7714
Epoch 2/20
318/318 ━━━━━━━━━━━━━━━━━━━━ 30s 94ms/step - accuracy: 0.6484 - loss: 0.6402 - val_accuracy: 0.6855 - val_loss: 0.6118
Epoch 3/20
318/318 ━━━━━━━━━━━━━━━━━━━━ 40s 125ms/step - accuracy: 0.6565 - loss: 0.6251 - val_accuracy: 0.6950 - val_loss: 0.6918
Epoch 4/20
318/318 ━━━━━━━━━━━━━━━━━━━━ 39s 122ms/step - accuracy: 0.6509 - loss: 0.6306 - val_accuracy: 0.5220 - val_loss: 0.7452
Epoch 5/20
318/318 ━━━━━━━━━━━━━━━━━━━━ 30s 94ms/step - accuracy: 0.6556 - loss: 0.6110 - val_accuracy: 0.6855 - val_loss: 0.6303
Epoch 6/20
318/318 ━━━━━━━━━━━━━━━━━━━━ 30s 96ms/step - accuracy: 0.7076 - loss: 0.6012 - val_accuracy: 0.6730 - val_loss: 0.7881
Epoch 7/20
318/318 ━━━━━━━━━━━━━━━━━━━━ 31s 96ms/step - accuracy: 0.6852 - loss: 0.6016 - val_accuracy: 0.7044 - val_loss: 0.5916
Epoch 8/20
318/318 ━━━━━━━━━━━━━━━━━━━━ 41s 97ms/step - accuracy: 0.6940 - loss: 0.6082 -

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to model's input size
])

In [ ]:
# Define directories
source_folder = "/content/drive/MyDrive/Image/Images"
informative_folder = "Informative"
non_informative_folder = "Non_Informative"

# Create output directories if they don't exist
os.makedirs(informative_folder, exist_ok=True)
os.makedirs(non_informative_folder, exist_ok=True)


In [ ]:



def preprocess_image(img_path, target_size=(224, 224)):
    img = load_img(img_path, target_size=target_size)  # Resize
    img_array = img_to_array(img)  # Convert to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to [0,1]
    return img_array


In [ ]:
for img_name in os.listdir(source_folder):
    img_path = os.path.join(source_folder, img_name)

    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):  # Skip non-image files
        continue

    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)[0][0]  # Get probability

    if prediction >= 0.5:  # Threshold: 0.5
        shutil.move(img_path, os.path.join(informative_folder, img_name))
    else:
        shutil.move(img_path, os.path.join(non_informative_folder, img_name))

print("Images sorted successfully!")


NameError: name 'os' is not defined